<a href="https://colab.research.google.com/github/tantar/hack/blob/main/Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run streamlit app from a Google Colab Notebook
To set-up the first step is to run the below code cell, then you must immeadiately restart the runtime by going to Runtime>Restart Runtime


In [1]:
!pip install -q streamlit

     |████████████████████████████████| 8.2MB 4.3MB/s 
     |████████████████████████████████| 4.2MB 40.1MB/s 
     |████████████████████████████████| 81kB 8.5MB/s 
     |████████████████████████████████| 163kB 44.8MB/s 
     |████████████████████████████████| 112kB 43.8MB/s 
     |████████████████████████████████| 122kB 40.8MB/s 
     |████████████████████████████████| 71kB 7.3MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.4 which is incompatible.


Reset the execution environment after streamlit installation

## Download a streamlit app example

This example comes from [How to Build a Simple Machine Learning Web App in Python](https://www.youtube.com/watch?v=8M20LyCZDOY) 

In [1]:
#The first step is to mount google drive to the python environment allowing access to the data
from google.colab import drive
drive.mount('/content/gdrive')
datafolder = f'/content/gdrive/MyDrive/Hackathon/data'
workfolder = f'/content/gdrive/MyDrive/Hackathon/work'

import streamlit as st
import pandas as pd
import numpy as np
import datetime as dt
import os
import altair as alt

os.chdir(workfolder)\

# files = os.listdir(datafolder)
# data = pd.DataFrame()

# for file in files:
#   dat = pd.read_csv("data/" + file)
#   f_name = file.split("p")[0]
#   dat["participant_id"] = (f_name + dat.participant_id.astype(str))
#   data = data.append(dat)

cols = ["participant_id","visit_name","hoehn_and_yahr_stage","age_at_baseline","age_at_diagnosis","date_visit","date_baseline"]

d = pd.read_csv("data/PPMI_processed.csv", usecols = cols)

d.loc[:,"BL_durdx"] = d.age_at_baseline - d.age_at_diagnosis
d.loc[:,"daysB"] = d.date_visit - d.date_baseline
d.loc[:,"daysDx"] = (d.daysB + d.BL_durdx * 365.25).round(2)
d.loc[:,"yearsDx"] = (d.daysDx / 365.25).round(0)

ydx = d.loc[:,["participant_id","yearsDx","hoehn_and_yahr_stage"]].dropna().drop_duplicates(keep="first", subset = ["participant_id","yearsDx"])
ydx = ydx[(ydx.yearsDx > 0)]
#ydx = ydx.pivot_table(columns = "hoehn_and_yahr_stage", index = "yearsDx", values = 'participant_id', aggfunc = "count").fillna(0)
ydx.loc[:,"hoehn_and_yahr_stage"] = ydx.hoehn_and_yahr_stage.round()
ydx.loc[:,"HY Stage Greater Than 3"] = ["HY<3" if i < 3 
                      else "HY>=3" for i in ydx.hoehn_and_yahr_stage]
ydx.loc[:,"yearsDx"] = (ydx.yearsDx / 2.5).round(0) * 2.5

c1 = alt.Chart(ydx).mark_bar(size = 20).encode(
     y='yearsDx',
     x=alt.X("count():Q", stack="normalize"),
     color='hoehn_and_yahr_stage:N'
)

ydx = ydx.rename({"yearsDx" : "Years Since Diagnosis"},axis=1)

c2 = alt.Chart(ydx).mark_bar(size = 20).encode(
     x="Years Since Diagnosis",
     y=alt.Y("count():Q", stack="normalize"),
     color='HY Stage Greater Than 3:N'
)

c2

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


2021-05-10 17:23:14.781 INFO    numexpr.utils: NumExpr defaulting to 2 threads.


alt.Chart(...)

In [2]:
from google.colab import drive
import streamlit as st
import pandas as pd
import numpy as np
import datetime as dt
import os
import altair as alt


drive.mount('/content/gdrive')
datafolder = f'/content/gdrive/MyDrive/Hackathon/data'
workfolder = f'/content/gdrive/MyDrive/Hackathon/work'

os.chdir(workfolder)\

cols = ["participant_id","visit_name", "Phenotype","hoehn_and_yahr_stage","mds_updrs_part_i_summary_score",
        "mds_updrs_part_ii_summary_score","mds_updrs_part_iii_summary_score","mds_updrs_part_iv_summary_score",
        "age_at_baseline","age_at_diagnosis","date_visit","date_baseline"]

dataset = st.selectbox("Pick a Dataset", ["PPMI","PDBP","All"])

if dataset == "PPMI":
  d = pd.read_csv("data/PPMI_processed.csv", usecols = cols)
if dataset == "PDBP":
  d = pd.read_csv("data/PDBP_processed.csv", usecols = cols)
if dataset == "All":
  d1 = pd.read_csv("data/PPMI_processed.csv", usecols = cols)
  d2 = pd.read_csv("data/PDBP_processed.csv", usecols = cols)
  d = d1.append(d2)

d = d.rename({"hoehn_and_yahr_stage" : "HY","mds_updrs_part_i_summary_score" : "UPDRS1",
        "mds_updrs_part_ii_summary_score":"UPDRS2","mds_updrs_part_iii_summary_score":"UPDRS3",
        "mds_updrs_part_iv_summary_score":"UPDRS4"}, axis = 1)

if st.checkbox("Include control"):
  d = d[d.Phenotype == "PD"]

d.loc[:,"BL_durdx"] = d.age_at_baseline - d.age_at_diagnosis
d.loc[:,"daysB"] = d.date_visit - d.date_baseline
d.loc[:,"daysDx"] = (d.daysB + d.BL_durdx * 365.25).round(2)
d.loc[:,"yearsDx"] = (d.daysDx / 365.25).round(0)


ydx = d.copy()
slice_width = st.slider("Time Interval", 1.0, 7.5, 2.5, step = 0.25)
ydx.loc[:,"yearsDx"] = (ydx.yearsDx / slice_width).round(0) * slice_width
ydx = ydx.rename({"yearsDx" : "Years Since Diagnosis"},axis=1)

c2 = alt.Chart(ydx).mark_bar(size = slice_width * (7.5 - slice_width/10)).encode(
     x="Years Since Diagnosis",
     y=alt.Y("count():Q", stack="normalize"),
     color='HY Stage Greater Than 3:N'
).properties(width=750)


st.title("Hackathon Proof of Concept")

if st.checkbox("Show Raw Data"):
  st.subheader("Raw data")
  st.write(d)

st.altair_chart(c2)

2021-05-10 17:23:17.344 WARNING root: 
  command:

    streamlit run /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py [ARGUMENTS]


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [13]:
from google.colab import drive
import streamlit as st
import pandas as pd
import numpy as np
import datetime as dt
import os
import altair as alt
import plotly.graph_objects as go

drive.mount('/content/gdrive')
datafolder = f'/content/gdrive/MyDrive/Hackathon/data'
workfolder = f'/content/gdrive/MyDrive/Hackathon/work'

os.chdir(workfolder)\

cols = ["participant_id","visit_name", "Phenotype","mds_updrs_part_i_summary_score",
        "mds_updrs_part_ii_summary_score","mds_updrs_part_iii_summary_score","mds_updrs_part_iv_summary_score",
        "age_at_baseline","age_at_diagnosis","date_visit","date_baseline"]

d1 = pd.read_csv("/content/gdrive/MyDrive/Hackathon/data/PPMI_processed.csv", usecols = cols)
d2 = pd.read_csv("/content/gdrive/MyDrive/Hackathon/data/PDBP_processed.csv", usecols = cols)
d3 = pd.read_csv("/content/gdrive/MyDrive/Hackathon/data/BIOFIND_processed.csv", usecols = cols)
d = d1.append(d2).append(d3)

d = d.rename({"hoehn_and_yahr_stage" : "HY","mds_updrs_part_i_summary_score" : "UPDRS1",
        "mds_updrs_part_ii_summary_score":"UPDRS2","mds_updrs_part_iii_summary_score":"UPDRS3",
        "mds_updrs_part_iv_summary_score":"UPDRS4"}, axis = 1)
d = d[d.Phenotype == "PD"]

d.loc[:,"BL_durdx"] = d.age_at_baseline - d.age_at_diagnosis
d.loc[:,"daysB"] = d.date_visit - d.date_baseline
d.loc[:,"daysDx"] = (d.daysB + d.BL_durdx * 365.25).round(2)
d.loc[:,"yearsDx"] = (d.daysDx / 365.25).round(0)

ny = 10

d = d[(d.yearsDx >= 0) & (d.yearsDx <= ny)]
yearsDx = "Years Since Diagnosis"

ydx = d.copy()#.loc[:,["participant_id",yearsDx,"HY"]].dropna().drop_duplicates(keep="first", subset = ["participant_id",yearsDx])
ydx.loc[:,"HY"] = ydx.HY.round()
ydx.loc[:,"HY3"] = ["0" if i < 3 
                      else "1" for i in ydx.HY]



SNK = ydx.pivot_table(values = "HY3", index = "participant_id", columns = "yearsDx", aggfunc = 'max').astype(float)
SNK.columns = range(0,ny+1)

count = SNK.reset_index().pivot_table(values = "participant_id", columns = range(0,ny+1), aggfunc = "count", dropna = False)

count
# count = count.melt()

# MC2 = pd.DataFrame()
# nl = 3
# for i in range(0,ny):
#   MC = count.iloc[:,[i,(i+1),ny+1]].groupby([i,(i+1)]).agg(sum).reset_index()
#   MC.iloc[:,1] = [int(x)+(nl*i)+nl for x in list(MC.iloc[:,1])]
#   if i > 0:
#     MC.iloc[:,0] = [int(x)+(nl*i) for x in list(MC.iloc[:,0])]
#   MC.columns = ["SRC","TRGT","VAL"]
#   MC2 = MC2.append(MC)

# MC2

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.

2021-05-10 18:17:06.412 INFO    numexpr.utils: NumExpr defaulting to 2 threads.


AttributeError: ignored

In [12]:
BF = pd.read_csv("/content/gdrive/MyDrive/Hackathon/data/BIOFIND_processed.csv"

,participant_id,visit_name,visit_month,date_visit,date_baseline,age_at_baseline,date_birth,sex,ethnicity,race,education_years,family_hx_1st,family_hx_2nd,family_hx_generations,family_hx_n_of_PD,study_arm,Phenotype,weight_kg,height_cm,BMI,heart_rate,systolic_blood_pressure,diastolic_blood_pressure,dx_essential_bradykinesia,dx_essential_rest_tremor,dx_essential_rigidity,primary_diagnosis,primary_diagnosis_text,age_at_onset,age_at_diagnosis,clinical_dx_constipation,clinical_dx_depression,clinical_dx_insomnia,hx_hypertension,hx_hyperlipidemia,hx_diabetes,hx_heart_disease,hx_dementia,hx_stroke,hx_depression,...,moca27_orientation_city_score,moca_orientation_subscore,moca_total_score,rbd_info_source,code_rbd01_vivid_dreams,code_rbd02_aggressive_or_action_packed_dreams,code_rbd03_nocturnal_behaviour,code_rbd04_move_arms_legs_during_sleep,code_rbd05_hurt_bed_partner,code_rbd06_1_speaking_in_sleep,code_rbd06_2_sudden_limb_movements,code_rbd06_3_complex_movements,code_rbd06_4_things_fell_down,code_rbd07_my_movements_awake_me,code_rbd08_remember_dreams,code_rbd09_sleep_is_disturbed,code_rbd10a_stroke,code_rbd10b_head_trauma,code_rbd10c_parkinsonism,code_rbd10d_rls,code_rbd10e_narcolepsy,code_rbd10f_depression,code_rbd10g_epilepsy,code_rbd10h_brain_inflammatory_disease,code_rbd10i_other,code_rbd10i_other_comment,code_rbd10_nervous_system_disease,rbd_summary_score,sbp_standing,dbp_standing,hr_standing,sbp_supine,dbp_supine,hr_supine,sbp_sitting,dbp_sitting,hr_sitting,mod_schwab_england_pct_adl_score,pdrfuq_smoke_100,pdrfuq_alcohol_regularly
0,1101,BL,NaN,15675,15675,74.915811,-11688,Female,Not Hispanic or Latino,Black or African American,13.0,No,No,No,0.0,PD,PD,59.3,163.0,22.319244,NaN,NaN,NaN,No,Yes,No,idiopathic PD,NaN,68.000000,68.413415,No,No,No,Yes,No,No,No,Yes,No,No,...,1.0,6.0,22.0,Patient and Caregiver,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,145.0,80.0,65.0,170.0,86.5,66.5,NaN,NaN,NaN,90.0,NaN,NaN
1,1001,BL,NaN,15737,15737,55.085558,-4383,Male,Not Hispanic or Latino,White,12.0,No,No,No,0.0,Control,Control,108.5,171.0,37.105434,NaN,NaN,NaN,NaN,NaN,NaN,No PD nor other neurological disorder,NaN,NaN,NaN,No,No,No,No,No,No,No,No,No,No,...,1.0,6.0,29.0,Patient,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,4.0,127.5,90.0,73.0,136.0,78.0,67.0,NaN,NaN,NaN,NaN,NaN,NaN
2,1102,BL,NaN,15737,15737,69.086927,-9497,Male,Not Hispanic or Latino,White,14.0,No,No,No,0.0,PD,PD,120.0,183.0,35.832661,NaN,NaN,NaN,Yes,Yes,Yes,idiopathic PD,NaN,59.915127,60.583162,No,No,No,No,No,Yes,No,No,No,No,...,1.0,6.0,27.0,Patient and Caregiver,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,4.0,126.0,70.5,78.5,157.5,80.0,71.0,NaN,NaN,NaN,80.0,NaN,NaN
3,1201,BL,NaN,15737,15737,80.084873,-13514,Female,Not Hispanic or Latino,White,16.0,No,No,No,0.0,PD,PD,58.5,160.0,22.851562,NaN,NaN,NaN,Yes,Yes,Yes,idiopathic PD,NaN,68.000000,68.000000,No,Yes,No,No,No,No,No,No,No,Yes,...,1.0,6.0,26.0,Patient,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,1.0,7.0,116.0,70.5,74.5,138.5,74.0,65.0,NaN,NaN,NaN,80.0,NaN,NaN
4,1002,BL,NaN,15765,15765,66.162902,-8401,Female,Not Hispanic or Latino,White,16.0,No,No,No,0.0,PD,PD,72.1,167.0,25.852487,NaN,NaN,NaN,Yes,No,Yes,idiopathic PD,NaN,60.999316,61.084189,No,No,No,No,No,No,No,No,No,No,...,1.0,6.0,29.0,Patient,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,4.0,107.5,67.0,65.0,107.5,66.0,62.0,NaN,NaN,NaN,100.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,1200,BL,NaN,16495,16495,69.160849,-8766,Female,Not Hispanic or Latino,White,2

In [1]:
import plotly.graph_objects as go

cols = ["participant_id","visit_name", "Phenotype","hoehn_and_yahr_stage","mds_updrs_part_i_summary_score",
        "mds_updrs_part_ii_summary_score","mds_updrs_part_iii_summary_score","mds_updrs_part_iv_summary_score",
        "age_at_baseline","age_at_diagnosis","date_visit","date_baseline"]

d1 = pd.read_csv("data/PPMI_processed.csv", usecols = cols)
d2 = pd.read_csv("data/PDBP_processed.csv", usecols = cols)
d = d1.append(d2)

d = d.rename({"hoehn_and_yahr_stage" : "HY","mds_updrs_part_i_summary_score" : "UPDRS1",
        "mds_updrs_part_ii_summary_score":"UPDRS2","mds_updrs_part_iii_summary_score":"UPDRS3",
        "mds_updrs_part_iv_summary_score":"UPDRS4"}, axis = 1)
d = d[d.Phenotype == "PD"]

d.loc[:,"BL_durdx"] = d.age_at_baseline - d.age_at_diagnosis
d.loc[:,"daysB"] = d.date_visit - d.date_baseline
d.loc[:,"daysDx"] = (d.daysB + d.BL_durdx * 365.25).round(2)
d.loc[:,"yearsDx"] = (d.daysDx / 365.25).round(0)

ny = 10

d = d[(d.yearsDx >= 0) & (d.yearsDx <= ny)]
yearsDx = "Years Since Diagnosis"

ydx = d.copy()#.loc[:,["participant_id",yearsDx,"HY"]].dropna().drop_duplicates(keep="first", subset = ["participant_id",yearsDx])
ydx.loc[:,"HY"] = ydx.HY.round()
ydx.loc[:,"HY3"] = ["0" if i < 3 
                      else "1" for i in ydx.HY]

SNK = ydx.pivot_table(values = "HY3", index = "participant_id", columns = "yearsDx", aggfunc = 'max')
SNK.columns = range(0,ny+1)
SNK = SNK.fillna(2)
nl = 3

count = SNK.reset_index().pivot_table(values = "participant_id", columns = range(0,ny+1), aggfunc = "count")

count = count.melt()

nl = 3

MC2 = pd.DataFrame()
for i in range(0,ny):
  MC = count.iloc[:,[i,(i+1),ny+1]].groupby([i,(i+1)]).agg(sum).reset_index()
  MC.columns = ["SRC","TRGT","VAL"]
  MC = MC.astype(int)
  #MC = MC[MC.SRC != MC.TRGT].sort_values(["SRC","TRGT"])
  MC.iloc[:,1] = [int(x)+(nl*i)+nl for x in list(MC.iloc[:,1])]
  if i > 0:
    MC.iloc[:,0] = [int(x)+(nl*i) for x in list(MC.iloc[:,0])]
  MC2 = MC2.append(MC)

MC2 = MC2.sort_values("SRC")

if nl == 2:
  labl = ["HY<3", "HY>=3"]* int(len(MC2) / nl)
elif nl == 3:
  labl = ["HY<3", "HY>=3", "CENS"] * int(len(MC2) / nl)

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = labl,
      color = "blue"
    ),
    link = dict(
      source = np.array(MC2.SRC), # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = np.array(MC2.TRGT),
      value = np.array(MC2.VAL)
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()

NameError: ignored

In [16]:
d = pd.read_csv("data/BIOFIND_processed.csv", usecols = cols)
d

FileNotFoundError: ignored

In [12]:
%%writefile app.py
from google.colab import drive
import streamlit as st
import pandas as pd
import numpy as np
import datetime as dt
import os
import altair as alt
import plotly.graph_objects as go

drive.mount('/content/gdrive')
datafolder = f'/content/gdrive/MyDrive/Hackathon/data'
workfolder = f'/content/gdrive/MyDrive/Hackathon/work'

os.chdir(workfolder)\


#Set up the general format for the app 
st.title("Hackathon Proof of Concept")
st.sidebar.header("Variable of Interest")

#Creating checkboxes and slide bars
dataset = st.selectbox("Pick a Dataset", ["PPMI","PDBP","BIOFIND","All"])
viz = st.sidebar.selectbox("Pick a Visualization Type", ["Bar Graph", "Sankey Graph"])
var = st.sidebar.selectbox("Pick a Variable", ["HY","UPDRS1","UPDRS2","UPDRS3","UPDRS4"])

cols = ["participant_id","visit_name", "Phenotype","hoehn_and_yahr_stage","mds_updrs_part_i_summary_score",
        "mds_updrs_part_ii_summary_score","mds_updrs_part_iii_summary_score","mds_updrs_part_iv_summary_score",
        "age_at_baseline","age_at_diagnosis","date_visit","date_baseline"]

if dataset == "PPMI":
  d = pd.read_csv("data/PPMI_processed.csv", usecols = cols)
if dataset == "PDBP":
  d = pd.read_csv("data/PDBP_processed.csv", usecols = cols)
if dataset == "BIOFIND":
  d = pd.read_csv("data/BIOFIND_processed.csv", usecols = cols)
if dataset == "All":
  d1 = pd.read_csv("data/PPMI_processed.csv", usecols = cols)
  d2 = pd.read_csv("data/PDBP_processed.csv", usecols = cols)
  d3 = pd.read_csv("data/BIOFIND_processed.csv", usecols = cols)
  d = d1.append(d2).append(d3)

d = d.rename({"hoehn_and_yahr_stage" : "HY","mds_updrs_part_i_summary_score" : "UPDRS1",
        "mds_updrs_part_ii_summary_score":"UPDRS2","mds_updrs_part_iii_summary_score":"UPDRS3",
        "mds_updrs_part_iv_summary_score":"UPDRS4"}, axis = 1)
d = d[d.Phenotype == "PD"]

d.loc[:,"BL_durdx"] = d.age_at_baseline - d.age_at_diagnosis
d.loc[:,"daysB"] = d.date_visit - d.date_baseline
d.loc[:,"daysDx"] = (d.daysB + d.BL_durdx * 365.25).round(2)
d.loc[:,"yearsDx"] = (d.daysDx / 365.25).round(0)


if viz == "Bar Graph":
  d = d[(d.yearsDx > 0)]
  yearsDx = "Years Since Diagnosis"

  slice_width = st.slider("Time Interval", 1.0, 7.5, 2.5, step = 0.25)
  d.loc[:,"yearsDx"] = (d.yearsDx / slice_width).round(0) * slice_width
  d = d.rename({"yearsDx" : "Years Since Diagnosis"},axis=1)
  size = slice_width * (7.5 - slice_width/10)
  if var == "HY":
    ydx = d.loc[:,["participant_id",yearsDx,"HY"]].dropna().drop_duplicates(keep="first", subset = ["participant_id",yearsDx])
    ydx.loc[:,"HY"] = ydx.HY.round()
    ydx.loc[:,"HY Stage Greater Than 3"] = ["HY<3" if i < 3 
                          else "HY>=3" for i in ydx.HY]

    c2 = alt.Chart(ydx).mark_bar(size = size).encode(
      x="Years Since Diagnosis",
      y=alt.Y("count():Q", stack="normalize"),
      color='HY Stage Greater Than 3:N'
    ).properties(width=750)

    st.altair_chart(c2)
  elif var == "UPDRS1":
    ydx = d.loc[:,["participant_id",yearsDx,"UPDRS1"]].dropna().drop_duplicates(keep="first", subset = ["participant_id",yearsDx])
    ydx.loc[:,"UPDRS Part 1 Score"] = [(var + "<10")  if i < 10 
                        else (var + "<20") if i < 20
                        else (var + "<30")  if i < 30
                        else (var + "<40")  if i< 40
                        else (var + "<50")  if i< 50
                        else np.nan for i in ydx.loc[:,var]]

    c2 = alt.Chart(ydx).mark_bar(size = size).encode(
      x="Years Since Diagnosis",
      y=alt.Y("count():Q", stack="normalize"),
      color='UPDRS Part 1 Score:N'
    ).properties(width=750)

    st.altair_chart(c2)
  elif var == "UPDRS2":
    ydx = d.loc[:,["participant_id",yearsDx,"UPDRS2"]].dropna().drop_duplicates(keep="first", subset = ["participant_id",yearsDx])
    ydx.loc[:,"UPDRS Part 2 Score"] = [(var + "<10")  if i < 10 
                        else (var + "<20") if i < 20
                        else (var + "<30")  if i < 30
                        else (var + "<40")  if i< 40
                        else (var + "<50")  if i< 50
                        else np.nan for i in ydx.loc[:,var]]
    
    c2 = alt.Chart(ydx).mark_bar(size = size).encode(
      x="Years Since Diagnosis",
      y=alt.Y("count():Q", stack="normalize"),
      color='UPDRS Part 2 Score:N'
    ).properties(width=750)

    st.altair_chart(c2)
  elif var == "UPDRS3":
    ydx = d.loc[:,["participant_id",yearsDx,"UPDRS3"]].dropna().drop_duplicates(keep="first", subset = ["participant_id",yearsDx])
    ydx.loc[:,"UPDRS Part 3 Score"] = [(var + "<10")  if i < 10 
                        else (var + "<20") if i < 20
                        else (var + "<30")  if i < 30
                        else (var + "<40")  if i< 40
                        else (var + "<50")  if i< 50
                        else np.nan for i in ydx.loc[:,var]]

    c2 = alt.Chart(ydx).mark_bar(size = size).encode(
      x="Years Since Diagnosis",
      y=alt.Y("count():Q", stack="normalize"),
      color='UPDRS Part 3 Score:N'
    ).properties(width=750)

    st.altair_chart(c2)
  elif var == "UPDRS4":
    ydx = d.loc[:,["participant_id",yearsDx,"UPDRS4"]].dropna().drop_duplicates(keep="first", subset = ["participant_id",yearsDx])
    ydx.loc[:,"UPDRS Part 4 Score"] = [(var + "<10")  if i < 10 
                        else (var + "<20") if i < 20
                        else (var + "<30")  if i < 30
                        else (var + "<40")  if i< 40
                        else (var + "<50")  if i< 50
                        else np.nan for i in ydx.loc[:,var]]

    c2 = alt.Chart(ydx).mark_bar(size = size).encode(
      x="Years Since Diagnosis",
      y=alt.Y("count():Q", stack="normalize"),
      color='UPDRS Part 4 Score:N'
    ).properties(width=750)

    st.altair_chart(c2)

if viz == "Sankey Graph":
  ny = st.slider("Select Number of Years Since Diagnosis", 1, 10, 5, step = 1)
  d = d[(d.yearsDx >= 0) & (d.yearsDx <= ny)]

  ydx = d.copy()#.loc[:,["participant_id",yearsDx,"HY"]].dropna().drop_duplicates(keep="first", subset = ["participant_id",yearsDx])
  ydx.loc[:,"HY"] = ydx.HY.round()
  ydx.loc[:,"HY3"] = ["0" if i < 3 
                        else "1" for i in ydx.HY]

  SNK = ydx.pivot_table(values = "HY3", index = "participant_id", columns = "yearsDx", aggfunc = 'max')
  SNK.columns = range(0,ny+1)

  nl = 2

  miss = st.selectbox("How are missing values handled?", ["Include Censoring", "Interpolate Values","Remove"])

  if miss == "Include Censoring":
    SNK = SNK.fillna(2)
    nl = 3
  if miss == "Interpolate Values":
    SNK = SNK.fillna(method = 'pad', axis = 1).fillna(method = 'bfill', axis = 1)
    nl = 2


  count = SNK.reset_index().pivot_table(values = "participant_id", columns = range(0,ny+1), aggfunc = "count")

  count = count.melt()

  MC2 = pd.DataFrame()
  for i in range(0,ny):
    MC = count.iloc[:,[i,(i+1),ny+1]].groupby([i,(i+1)]).agg(sum).reset_index()
    MC.columns = ["SRC","TRGT","VAL"]
    MC = MC.astype(int)
    MC.iloc[:,1] = [int(x)+(nl*i)+nl for x in list(MC.iloc[:,1])]
    if i > 0:
      MC.iloc[:,0] = [int(x)+(nl*i) for x in list(MC.iloc[:,0])]
    MC2 = MC2.append(MC)

  MC2 = MC2.sort_values(["SRC","TRGT"])

  if nl == 2:
    labl = ["HY<3", "HY>=3"]* int(len(MC2) / nl)
    node_y = [0,1]* int(len(MC2) / nl)
  elif nl == 3:
    labl = ["HY<3", "HY>=3", "CENS"]* int(len(MC2) / nl)
    node_y = [0,1,2]* int(len(MC2) / nl)

  fig = go.Figure(data=[go.Sankey(
      node = dict(
        pad = 15,
        thickness = 20,
        line = dict(color = "black", width = 0.5),
        label = labl,
        y = node_y,
        color = "blue"
      ),
      link = dict(
        source = np.array(MC2.SRC), # indices correspond to labels, eg A1, A2, A1, B1, ...
        target = np.array(MC2.TRGT),
        value = np.array(MC2.VAL)
    ))])
  
  fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
  st.write(fig)
  


Overwriting app.py


In [ ]:
MC2

In [ ]:
"d = pd.read_csv("data/PPMI_processed.csv")
d.Phenotype.unique()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (18,49) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


array(['Control', 'PD', 'Other'], dtype=object)

In [ ]:
!python3 "app.py"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
2021-04-29 14:58:08.450 WARNING root: 
  command:

    streamlit run app.py [ARGUMENTS]
2021-04-29 14:58:08.711 INFO    numexpr.utils: NumExpr defaulting to 2 threads.


## Install ngrok

In [7]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2021-05-10 17:23:45--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.204.190.140, 54.225.197.119, 34.202.43.88, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.204.190.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.6’

ngrok-stable-linux- 100%[===================>]  13.19M  29.3MB/s    in 0.5s    

2021-05-10 17:23:46 (29.3 MB/s) - ‘ngrok-stable-linux-amd64.zip.6’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ngrok                   


In [8]:
get_ipython().system_raw('./ngrok http 8501 &')

In [9]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: http://0f57eec19d5d.ngrok.io


In [10]:
!streamlit run app.py &>/dev/null&#Sends to background

In [ ]:
!pgrep streamlit

233


In [ ]:
##Change number depending on previous output
